# Hydrographic obs from the M120 cruise and from NEMO

In [ ]:
# parameters

esm_vfc_data_dir = "../esm-vfc-data/"
catalog_url = "https://raw.githubusercontent.com/ESM-VFC/esm-vfc-catalogs/master/catalogs/NEMO_ORCA05_FOCI_Test_Minimal.yaml"
obs_data_doi = "10.1594/PANGAEA.868654"

## Tech preamble

In [ ]:
# set up intake catalog
import intake
from esmvfc_cattools import fetch_zenodo_data
import os

os.environ["ESM_VFC_DATA_DIR"] = esm_vfc_data_dir

In [ ]:
# be able to load data directly from Pangaea.de
from pangaeapy.pandataset import PanDataSet

In [ ]:
# set up plotting
import hvplot.pandas
import hvplot.xarray
import geoviews.feature as gf
from cartopy import crs

## Get obs data, extract near-surface measurements, plot positions

In [ ]:
obs_data = PanDataSet(obs_data_doi)
obs_df = obs_data.data  # Pandas dataframe

In [ ]:
display(obs_data.title)
display(obs_df.head())

In [ ]:
near_surface_obs = obs_df.loc[
    obs_df.groupby("Event")["Depth water"].idxmin()
]

In [ ]:
(
    near_surface_obs.hvplot("Longitude", "Latitude", geo=True, kind="points")
    * gf.coastline
)

## Load catalog and fetch data

In [ ]:
model_data_cat = intake.open_catalog(catalog_url)
fetch_zenodo_data(
    model_data_cat["NEMO_ORCA05_FOCI_Test_Minimal_grid_T"]
)

## Restrict to Atlantic, calc mean SST, plot with obs positions

In [ ]:
model_dataset = model_data_cat["NEMO_ORCA05_FOCI_Test_Minimal_grid_T"].to_dask()
model_dataset = model_dataset.set_coords(["nav_lat", "nav_lon"])

# restrict to equatorial Atlantic
# this helps getting rid of non-monotonicity of the coord fields
model_dataset = model_dataset.isel(x=slice(410, 620), y=slice(150, 350))

# need compute / cast to numpy array here in order for datashade to work
# (see https://datashader.org/user_guide/Performance.html)
model_mean_sst = model_dataset.sosstsst.mean("time_counter").compute()

In [ ]:
(
    model_mean_sst.hvplot.quadmesh(
        "nav_lon", "nav_lat",
        geo=True, datashade=True)
    * near_surface_obs.hvplot(
        "Longitude", "Latitude",
        geo=True, kind="points", color="red")
    * gf.coastline
)